In [1]:
import os
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.utils import dataset_utils

from _model_config import *
exec(f'from {MODEL_NAME} import test_dataset')

c:\Users\ptanmay143\miniconda3\envs\dev\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\ptanmay143\miniconda3\envs\dev\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issu

Python Version           :  3.8.16
TensorFlow Version       :  2.10.1
Keras Version            :  2.10.0
Librosa Version          :  0.10.1
Audiomentations Version  :  0.35.0
vram limit set to 4096MB
Found 7514 files belonging to 118 classes.
Class names:  ['Acanthiza chrysorrhoa', 'Acanthiza lineata', 'Acanthiza nana', 'Acanthiza pusilla', 'Acanthiza reguloides', 'Acanthiza uropygialis', 'Acanthorhynchus tenuirostris', 'Accipiter cirrocephalus', 'Aidemosyne modesta', 'Alauda arvensis', 'Anhinga novaehollandiae', 'Anthochaera phrygia', 'Antigone rubicunda', 'Artamus cinereus', 'Artamus cyanopterus', 'Artamus minor', 'Artamus superciliosus', 'Barnardius zonarius', 'Callocephalon fimbriatum', 'Calyptorhynchus banksii', 'Calyptorhynchus lathami', 'Capra Hircus', 'Carduelis carduelis', 'Carterornis leucotis', 'Cervus Unicolour', 'Ceyx azureus', 'Chenonetta jubata', 'Chlamydera nuchalis', 'Cincloramphus mathewsi', 'Cinclosoma punctatum', 'Cisticola exilis', 'Climacteris picumnus', 'Colluric

In [2]:
file_paths, labels, class_names = dataset_utils.index_directory(
    SC["AUDIO_DATA_DIRECTORY"],
    labels="inferred",
    formats=(".ogg", ".mp3", ".wav", ".flac"),
    class_names=None,
    shuffle=True,
    seed=42,
    follow_links=False,
)

Found 7514 files belonging to 118 classes.


In [3]:
# Load the saved models
def load_models(models_directory="models"):
    models = {}
    for model_name in os.listdir(models_directory):
        model_path = os.path.join(models_directory, model_name)
        if os.path.isdir(model_path):
            models[model_name] = tf.keras.models.load_model(model_path)
    return models

In [4]:
# Evaluate the models on the test dataset
def evaluate_models(models, test_dataset):
    metrics = {}
    for model_name, model in models.items():
        print(f"Evaluating model: {model_name}")
        evaluation = model.evaluate(test_dataset, return_dict=True)
        metrics[model_name] = evaluation
    return metrics

In [5]:
# Generate classification report and confusion matrix for each model
def generate_reports(models, test_dataset):
    reports = {}
    y_true = []
    y_pred = {}

    for sample, label in test_dataset.unbatch():
        y_true.append(np.argmax(label.numpy()))

    y_true = np.array(y_true)

    for model_name, model in models.items():
        print(f"Generating report for model: {model_name}")
        y_pred[model_name] = []
        for sample, label in test_dataset.unbatch():
            prediction = model.predict(tf.expand_dims(sample, 0))
            y_pred[model_name].append(np.argmax(prediction))

        y_pred[model_name] = np.array(y_pred[model_name])
        reports[model_name] = {
            "classification_report": classification_report(
                y_true, y_pred[model_name], labels=labels, target_names=class_names
            ),
            "confusion_matrix": confusion_matrix(y_true, y_pred[model_name]),
            "accuracy": accuracy_score(y_true, y_pred[model_name]),
        }

    return reports

In [6]:
# Load and evaluate the models
models_directory = "models"
models = load_models(models_directory)
metrics = evaluate_models(models, test_dataset)

Evaluating model: EfficientNetV2B0
5/5 [==============================] - 9s 86ms/step - loss: 0.3782 - accuracy: 0.9211
Evaluating model: InceptionV3
5/5 [==============================] - 3s 158ms/step - loss: 0.3999 - accuracy: 0.8947
Evaluating model: MobileNetV2
5/5 [==============================] - 2s 121ms/step - loss: 0.3784 - accuracy: 0.9079
Evaluating model: MobileNetV3Large
5/5 [==============================] - 2s 125ms/step - loss: 0.9077 - accuracy: 0.8158
Evaluating model: MobileNetV3Small
5/5 [==============================] - 2s 167ms/step - loss: 3.9104 - accuracy: 0.0526
Evaluating model: ResNet50V2
5/5 [==============================] - 2s 139ms/step - loss: 0.5725 - accuracy: 0.8684


In [7]:
reports = generate_reports(models, test_dataset)

Generating report for model: EfficientNetV2B0
1/1 [==============================] - 0s 26ms/step
Generating report for model: InceptionV3
1/1 [==============================] - 0s 28ms/step
Generating report for model: MobileNetV2
1/1 [==============================] - 0s 22ms/step
Generating report for model: MobileNetV3Large
1/1 [==============================] - 0s 25ms/step
Generating report for model: MobileNetV3Small
1/1 [==============================] - 0s 24ms/step
Generating report for model: ResNet50V2
1/1 [==============================] - 0s 27ms/step


In [8]:
# Print out the evaluation metrics
for model_name, metric in metrics.items():
    print(f"Metrics for {model_name}:")
    for metric_name, value in metric.items():
        print(f"  {metric_name}: {value}")
    print("\n")

Metrics for EfficientNetV2B0:
  loss: 0.3781679570674896
  accuracy: 0.9210526347160339


Metrics for InceptionV3:
  loss: 0.3999229371547699
  accuracy: 0.8947368264198303


Metrics for MobileNetV2:
  loss: 0.3783857822418213
  accuracy: 0.9078947305679321


Metrics for MobileNetV3Large:
  loss: 0.9076963663101196
  accuracy: 0.8157894611358643


Metrics for MobileNetV3Small:
  loss: 3.910449266433716
  accuracy: 0.05263157933950424


Metrics for ResNet50V2:
  loss: 0.5724804401397705
  accuracy: 0.8684210777282715




In [9]:
# Print out the classification reports and confusion matrices
for model_name, report in reports.items():
    print(f"Classification Report for {model_name}:")
    print(report["classification_report"])
    print("\nConfusion Matrix for {model_name}:")
    print(report["confusion_matrix"])
    print("\nAccuracy for {model_name}:")
    print(report["accuracy"])
    print("\n\n")

Classification Report for EfficientNetV2B0:
                              precision    recall  f1-score   support

       Acanthiza chrysorrhoa       1.00      1.00      1.00         2
           Acanthiza lineata       1.00      0.50      0.67         2
              Acanthiza nana       0.00      0.00      0.00         0
           Acanthiza pusilla       1.00      1.00      1.00         2
        Acanthiza reguloides       0.89      1.00      0.94         8
       Acanthiza uropygialis       1.00      1.00      1.00         2
Acanthorhynchus tenuirostris       1.00      1.00      1.00         1
     Accipiter cirrocephalus       1.00      1.00      1.00         2
          Aidemosyne modesta       1.00      1.00      1.00         4
             Alauda arvensis       1.00      0.33      0.50         3
     Anhinga novaehollandiae       0.89      1.00      0.94         8
         Anthochaera phrygia       0.00      0.00      0.00         0
          Antigone rubicunda       0.90      

In [12]:
# Optionally, save the reports to files
for model_name, report in reports.items():
    with open(f"model_reports/{model_name}_report.txt", "w") as f:
        f.write(f"Classification Report for {model_name}:\n")
        f.write(report["classification_report"])
        f.write("\nConfusion Matrix for {model_name}:\n")
        f.write(np.array2string(report["confusion_matrix"]))
        f.write("\nAccuracy for {model_name}:\n")
        f.write(str(report["accuracy"]))
        f.write("\n\n")